# Regression Exercise - Solutions

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv('cal_housing_clean.csv')

In [3]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [5]:
x_data = housing.drop(['medianHouseValue'],axis=1)

In [6]:
y_val = housing['medianHouseValue']

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler()

In [11]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [13]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [14]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [15]:
import tensorflow as tf

In [16]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [17]:
feat_cols = [ age,rooms,bedrooms,pop,households,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [18]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [27]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_model_dir': 'C:\\Users\\motos\\AppData\\Local\\Temp\\tmp9gilymkm', '_tf_random_seed': 1, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_session_config': None, '_keep_checkpoint_max': 5}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [20]:
model.train(input_fn=input_func,steps=25000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\motos\AppData\Local\Temp\tmp92jcktzl\model.ckpt.
INFO:tensorflow:step = 1, loss = 8.45692e+11
INFO:tensorflow:global_step/sec: 387.915
INFO:tensorflow:step = 101, loss = 7.0186e+11 (0.264 sec)
INFO:tensorflow:global_step/sec: 485.16
INFO:tensorflow:step = 201, loss = 2.71104e+11 (0.207 sec)
INFO:tensorflow:global_step/sec: 474.811
INFO:tensorflow:step = 301, loss = 7.1796e+11 (0.208 sec)
INFO:tensorflow:global_step/sec: 504.716
INFO:tensorflow:step = 401, loss = 5.80009e+11 (0.197 sec)
INFO:tensorflow:global_step/sec: 530.083
INFO:tensorflow:step = 501, loss = 7.12521e+11 (0.188 sec)
INFO:tensorflow:global_step/sec: 508.555
INFO:tensorflow:step = 601, loss = 8.17459e+11 (0.201 sec)
INFO:tensorflow:global_step/sec: 368.33
INFO:tensorflow:step = 701, loss = 4.31908e+11 (0.273 sec)
INFO:tensorflow:global_step/sec: 428.145
INFO:tensorflow:step = 801, loss = 4.1744e+11 (0.230 sec)
INFO:tensorf

INFO:tensorflow:global_step/sec: 502.186
INFO:tensorflow:step = 8101, loss = 6.828e+10 (0.200 sec)
INFO:tensorflow:global_step/sec: 505.987
INFO:tensorflow:step = 8201, loss = 1.61578e+11 (0.194 sec)
INFO:tensorflow:global_step/sec: 584.172
INFO:tensorflow:step = 8301, loss = 7.65442e+10 (0.173 sec)
INFO:tensorflow:global_step/sec: 511.151
INFO:tensorflow:step = 8401, loss = 9.1115e+10 (0.195 sec)
INFO:tensorflow:global_step/sec: 585.882
INFO:tensorflow:step = 8501, loss = 1.18016e+11 (0.173 sec)
INFO:tensorflow:global_step/sec: 509.851
INFO:tensorflow:step = 8601, loss = 1.18082e+11 (0.194 sec)
INFO:tensorflow:global_step/sec: 570.861
INFO:tensorflow:step = 8701, loss = 8.38568e+10 (0.176 sec)
INFO:tensorflow:global_step/sec: 592.814
INFO:tensorflow:step = 8801, loss = 9.98221e+10 (0.167 sec)
INFO:tensorflow:global_step/sec: 516.418
INFO:tensorflow:step = 8901, loss = 3.08169e+10 (0.194 sec)
INFO:tensorflow:global_step/sec: 553.513
INFO:tensorflow:step = 9001, loss = 7.93876e+10 (0.18

INFO:tensorflow:global_step/sec: 532.906
INFO:tensorflow:step = 16201, loss = 1.57444e+11 (0.189 sec)
INFO:tensorflow:global_step/sec: 543.012
INFO:tensorflow:step = 16301, loss = 8.08179e+10 (0.186 sec)
INFO:tensorflow:global_step/sec: 500.927
INFO:tensorflow:step = 16401, loss = 7.65962e+10 (0.200 sec)
INFO:tensorflow:global_step/sec: 543.01
INFO:tensorflow:step = 16501, loss = 4.16675e+10 (0.185 sec)
INFO:tensorflow:global_step/sec: 548.965
INFO:tensorflow:step = 16601, loss = 1.5447e+11 (0.181 sec)
INFO:tensorflow:global_step/sec: 459.567
INFO:tensorflow:step = 16701, loss = 4.77689e+10 (0.221 sec)
INFO:tensorflow:global_step/sec: 409.756
INFO:tensorflow:step = 16801, loss = 3.56089e+10 (0.243 sec)
INFO:tensorflow:global_step/sec: 483.993
INFO:tensorflow:step = 16901, loss = 8.55363e+10 (0.210 sec)
INFO:tensorflow:global_step/sec: 461.682
INFO:tensorflow:step = 17001, loss = 8.95521e+10 (0.212 sec)
INFO:tensorflow:global_step/sec: 463.824
INFO:tensorflow:step = 17101, loss = 7.7461

INFO:tensorflow:global_step/sec: 304.516
INFO:tensorflow:step = 24301, loss = 1.08646e+11 (0.331 sec)
INFO:tensorflow:global_step/sec: 524.529
INFO:tensorflow:step = 24401, loss = 8.17727e+10 (0.188 sec)
INFO:tensorflow:global_step/sec: 547.466
INFO:tensorflow:step = 24501, loss = 7.86331e+10 (0.184 sec)
INFO:tensorflow:global_step/sec: 509.852
INFO:tensorflow:step = 24601, loss = 2.83279e+10 (0.196 sec)
INFO:tensorflow:global_step/sec: 386.071
INFO:tensorflow:step = 24701, loss = 4.14735e+10 (0.265 sec)
INFO:tensorflow:global_step/sec: 409.757
INFO:tensorflow:step = 24801, loss = 4.5321e+10 (0.239 sec)
INFO:tensorflow:global_step/sec: 480.506
INFO:tensorflow:step = 24901, loss = 4.48969e+10 (0.211 sec)
INFO:tensorflow:Saving checkpoints for 25000 into C:\Users\motos\AppData\Local\Temp\tmp92jcktzl\model.ckpt.
INFO:tensorflow:Loss for final step: 4.75576e+10.


** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [21]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [22]:
pred_gen = model.predict(predict_input_func)

In [23]:
predictions = list(pred_gen)

INFO:tensorflow:Restoring parameters from C:\Users\motos\AppData\Local\Temp\tmp92jcktzl\model.ckpt-25000


** Calculate the RMSE. Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [24]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [25]:
from sklearn.metrics import mean_squared_error

In [26]:
mean_squared_error(y_test,final_preds)**0.5

97776.464465122641

# Great Job!